<a href="https://colab.research.google.com/github/yuanyuan233333/Mathematical-Finance/blob/main/Ex_session_binomial_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EXERCISE SESSION: BINOMIAL MODEL

A Stock A is available on the market at the current price of 12 Euros. in 4 months, such price may move up to 16 or down to 10 euros. In each of the next 4-months periods, the stock price may increase or decrease by the same percentage as in the previous period. The risk-free interest rate on the market is 4% per year.
Consider also a stock B whose price is the square of the price of stock A at any time.

(2) Compute the current price of a european put option on stock B with maturity 8 months and strike $K = 144$.

In [ ]:
# set the parameters

# stock A
s0 = 12 # initial price
u = 16/12 # up factor
d = 10/12 # down factor

s1_u = s0*u # price in case of up-move
s1_d = s0*d # price in case of down-move

In [ ]:
# stock B

# We use upper case variables to distinguish between stock A and stock B

S0 = 12**2 # initial price

S1_u = s1_u**2 # price in case of up-move
S1_d = s1_d**2 # price in case of down-move

U = S1_u/S0 # up factor
D = S1_d/S0 # down factor

In [ ]:
r = 0.04 # yearly interest rate
R = (1+r)**(1/3) - 1 # interest rate per 4-month period

K = 144 # strike price

To compute the put option price we need to compute first the payoffs of the put option at maturity and the risk neutral probabilites of the events.

In [ ]:
# maturity is in two periods (8 months)

# the possible outcomes are S0*U*U, S0*U*D, S0*D*U, S0*D*D
S0_UU = S0*U*U
S0_UD = S0*U*D
S0_DU = S0*D*U
S0_DD = S0*D*D

print(f'Possible T=2 outcomes: {[S0_UU, S0_UD, S0_DU, S0_DD]}')

Possible T=2 outcomes: [455.1111111111111, 177.77777777777777, 177.77777777777777, 69.44444444444444]


In [ ]:
# we compute the payoffs of the option

def put_payoff(st, k):
  return max(0, k-st)

payoff_uu = put_payoff(S0_UU, K)
payoff_ud = put_payoff(S0_UD, K)
payoff_du = put_payoff(S0_DU, K)
payoff_dd = put_payoff(S0_DD, K)

print(f'Possible T=2 payoffs: {[payoff_uu, payoff_ud, payoff_du, payoff_dd]}')

Possible T=2 payoffs: [0, 0, 0, 74.55555555555556]


Recall that the risk netrual probabilities are computed as
$$
\begin{cases}
q_u = \frac{(1+R) - d}{u - d}\\
q_d = \frac{u - (1+R)}{u - d}
\end{cases}
$$

In [ ]:
# compute the risk netrual probabilities

qu = (1 + R - D)/(U - D)
qd = 1 - qu

print(f'qu is {qu}, qd is {qd}')

qu is 0.294198424039138, qd is 0.705801575960862


In [ ]:
# check that the risk-neutral probabilities make the discounted price a martingale (up to some machine error)

print((1/(1+R))*(qu*S0*U + qd*S0*D))

143.99999999999997


In [ ]:
# compute the discounted expected payoff
P0 = (payoff_uu*qu*qu + payoff_ud*qu*qd + payoff_du*qd*qu + payoff_dd*qd*qd)*1/((1+R)**2)

print(f'The price of a european put option on Stock B with strike K={K} and maturity 8 months is: {P0}')

The price of a european put option on Stock B with strike K=144 and maturity 8 months is: 36.181760844671416


(3) Find the replicating strategy of the european put option and verify that the price of the replicating portfolio is in line with the result found in (2).

In [ ]:
# binomial tree for S0
#                                   PUT OPTION PAYOFF (K = 144)
#                                  |
#              / S0*U*U = 4096/9   |      0
#             /                    |
#    /(A) S0*U                     |
#   /         \                    |
# S0             S0*U*D = 1600/9   |      0
#   \         /                    |
#    \(B) S0*D                     |
#             \                    |
#              \ S0*D*D = 625/9    |     671/9

Starting at t=1, we place ourselves on each of the two nodes and we construct a portfolio that replicates the payoff at t=2 from that particular node.

Suppose we are at node (B). We wish to construct a strategy $h$ such that in case of up-movement the value of the portfolio is 0, and in case of down-movement the value is $\frac{671}{9}$.
Denoting by $(x,y)$ the number of bonds and stocks respectively, we have
$$
\begin{cases}
x(1+R) + y S_0 UD = 0 \\
x(1+R) + y S_0 D^2 = \frac{671}{9}
\end{cases}
$$
Solving the following linear system yields
$$
y = \frac{671}{9S_0D(D - U)}, \;\;\;\;\;\;\;\;
x = - \frac{y S_0 U D}{(1+R)}
$$

In [ ]:
y = payoff_dd/(S0*D*(D - U))
x = -(y * S0*U*D)/(1+R)

print(f'The replicating strategy in node B is: {(x,y)}')

The replicating strategy in node B is: (120.75846889073881, -0.6882051282051282)


In [ ]:
# check that indeed the strategy replicates the payoff

print(f'Payoff in case of up-movement:{x*(1+R) + y*S0*D*U}') # very close to zero
print(f'Payoff in case of down-movement:{x*(1+R) + y*S0*D*D}')

Payoff in case of up-movement:1.4210854715202004e-14
Payoff in case of down-movement:74.55555555555556


In [ ]:
# compute the value of the portfolio in B
V1_b = x + y * S0 * D
print(f'The value of the replicating portfolio in node B is: {V1_b}')

The value of the replicating portfolio in node B is: 51.937956070225994


Clearly, in node A, whatever the next movement of the underlying the payoff is zero and hence also the replicating portfolio will have a zero value.

We now compute the value of the replicating portfolio at the beginning of the trading period, t=0.
We wish to find $(x,y)$ such that
$$
\begin{cases}
x(1+R) + y S_0 U = 0 \\
x(1+R) + y S_0 D = 51.938
\end{cases}
$$

In [ ]:
y = V1_b/(S0*(D - U))
x = -(y * S0 * U)/(1 + R)

print(f'The replicating strategy in t = 0 is: {(x,y)}')

The replicating strategy in t = 0 is: (84.12448952488, -0.332935615834782)


In [ ]:
# compute the value of the repliocating portfolio in t=0
V0 = x + y * S0
print(f'The value of the replicating portfolio in t=0 is: {V0}')

The value of the replicating portfolio in t=0 is: 36.1817608446714


Which corresponds exactly with the value of the put option computed in point (2) of the exercise

(4) Compare the above put with a siilar put option on stock  A with strike $K=12$. Which is more expensive?

In [ ]:
# different strike
k = 12

# maturity is in two periods (8 months)

# the possible outcomes are s0*u*u, s0*u*d, s0*d*u, s0*d*d
s0_uu = s0*u*u
s0_ud = s0*u*d
s0_du = s0*d*u
s0_dd = s0*d*d

print(f'Possible T=2 outcomes: {[s0_uu, s0_ud, s0_du, s0_dd]}')

Possible T=2 outcomes: [21.333333333333332, 13.333333333333334, 13.333333333333332, 8.333333333333334]


In [ ]:
# compute the payoffs of the option

payoffA_uu = put_payoff(s0_uu, k)
payoffA_ud = put_payoff(s0_ud, k)
payoffA_du = put_payoff(s0_du, k)
payoffA_dd = put_payoff(s0_dd, k)

print(f'Possible T=2 payoffs: {[payoffA_uu, payoffA_ud, payoffA_du, payoffA_dd]}')

Possible T=2 payoffs: [0, 0, 0, 3.666666666666666]


In [ ]:
# we compute the risk netrual probabilities

quA = (1 + R - d)/(u - d)
qdA = 1 - quA

print(f'qu is {quA}, qd is {qdA}')

qu is 0.35965214097368775, qd is 0.6403478590263123


In [ ]:
# check that the risk-neutral probabilities make the discounted price a martingale

print((1/(1+R))*(quA*s0*u + qdA*s0*d))

12.0


In [ ]:
# we compute the discounted expected payoff
p0 = (payoffA_uu*quA*quA + payoffA_ud*quA*qdA + payoffA_du*qdA*quA + payoffA_dd*qdA*qdA)*1/((1+R)**2)

print(f'The price of a european put option on Stock A with strike K={k} and maturity 8 months is: {p0}')

The price of a european put option on Stock A with strike K=12 and maturity 8 months is: 1.4646970084540387


(5) Using the Put-Call Parity, deduce the prices of the corresponding European Call options.

Recall that the put-call parity is the following equality
$$
C_0 - P_0 = S_0 - K(1+R)^{-T}
$$
Therefore, given the price of a put option $P_0$ we can compute the price of the corresponding call option $C_0$ with the same underlying, same strike $K$, same maturity $T$ as
$$
C_0 = P_0 + S_0 - K(1+R)^{-T}
$$

In [ ]:
# compute the price of a call option on stock B with maturity 8 months and strike K=144 on stock B
C0 = P0 + S0 - K*((1+R)**(-2))
print(f'The price of a european call option on Stock B with strike K={K} and maturity 8 months is: {C0}')

The price of a european call option on Stock B with strike K=144 and maturity 8 months is: 39.89815108495458


In [ ]:
# compute the price of a call option on stock B with maturity 8 months and strike K=144 on stock B
c0 = p0 + s0 - k*((1+R)**(-2))
print(f'The price of a european call option on Stock A with strike K={k} and maturity 8 months is: {c0}')

The price of a european call option on Stock A with strike K=12 and maturity 8 months is: 1.7743961951443001
